Load libraries and create df

In [1]:
import pandas as pd
import os
import shutil
path = os.chdir('C:/Users/jewel.espiritu/OneDrive - AMDATEX LAS PINAS SERVICES INC/Python/py') 
sladf = pd.concat(pd.read_excel('./data/WeeklySLA_Data_2022.xlsx', sheet_name=None), ignore_index=True)

Create pivot table

In [2]:
SLApivot = pd.pivot_table(sladf, index=['Service','Week'], columns=['TTR Result\n(Excluding Pending Time)'], values=['Ticket No'], aggfunc='count')
SLApivot


Ticket No           
TTR Result
(Excluding Pending Time)                     Beyond TAT Within TAT
Service                                            Week                      
3rd Party/ Client Controlled Software, Network ... 1          70.0      245.0
                                                   2          89.0      272.0
                                                   3          69.0      143.0
                                                   4          57.0      197.0
                                                   5          71.0      135.0
...                                                            ...        ...
WFH PC setup                                       21         40.0       94.0
                                                   22         41.0       82.0
                                                   23         35.0       74.0
                                                   24         22.0       66.0
                                                   25         21.0       60.0

[441 rows x 2 columns]

Drop the unwanted column level of the pivot table

In [3]:
SLApivot.columns = SLApivot.columns.droplevel(0)
SLApivot.columns.name = None
newdf = SLApivot.reset_index()
newdf

,Service,Week,Beyond TAT,Within TAT
0,"3rd Party/ Client Controlled Software, Network...",1,70.0,245.0
1,"3rd Party/ Client Controlled Software, Network...",2,89.0,272.0
2,"3rd Party/ Client Controlled Software, Network...",3,69.0,143.0
3,"3rd Party/ Client Controlled Software, Network...",4,57.0,197.0
4,"3rd Party/ Client Controlled Software, Network...",5,71.0,135.0
...,...,...,...,...
436,WFH PC setup,21,40.0,94.0
437,WFH PC setup,22,41.0,82.0
438,WFH PC setup,23,35.0,74.0
439,WFH PC setup,24,22.0,66.0


Remove NA and replace with 0. Then remove decimal point

In [4]:
newdf['Beyond TAT'] = newdf['Beyond TAT'].fillna(0)
newdf['Within TAT'] = newdf['Within TAT'].fillna(0)

newdf['Beyond TAT'] = newdf['Beyond TAT'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['Within TAT'] = newdf['Within TAT'].astype(str).apply(lambda x: x.replace('.0',''))

Filter the current week

In [5]:
newdf = newdf.query('Week == 26')

Compute the percentage of beyond TAT

In [6]:
sumbeyondtat = newdf['Beyond TAT'].astype(int).sum()
beyondpercent = newdf['Beyond TAT'].astype(int)/sumbeyondtat
newdf["Beyond TAT Percent"] = beyondpercent

C:\Users\jewel.espiritu\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Rearrange the data using value of beyond TAT

In [7]:
newdf = newdf.rename( columns = {'Beyond TAT': 'Beyond_TAT'}, inplace = False)
newdf.Beyond_TAT = newdf.Beyond_TAT.astype(float)
beyond = "Beyond_TAT"
newdf = newdf.sort_values(beyond, ascending=False)

Create a column for cumulative sum and cumulative percentage

In [8]:
newdf['Beyond_TAT'] = newdf['Beyond_TAT'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['cumulative_sum'] = newdf[beyond].astype(int).cumsum()
newdf['cumulative_perc'] = 100*newdf['cumulative_sum']/newdf[beyond].astype(int).sum()
newdf

,Service,Week,Beyond_TAT,Within TAT,Beyond TAT Percent,cumulative_sum,cumulative_perc
99,Hardware,25,119,287,0.260965,119,26.096491
49,Access,25,108,203,0.236842,227,49.780702
390,Software,25,63,159,0.138158,290,63.596491
24,"3rd Party/ Client Controlled Software, Network...",25,45,282,0.098684,335,73.464912
74,Email,25,44,66,0.096491,379,83.114035
311,Resignee's Clearance Processing,25,27,36,0.059211,406,89.035088
440,WFH PC setup,25,21,60,0.046053,427,93.640351
224,Network or internet,25,8,9,0.017544,435,95.394737
341,Server,25,8,20,0.017544,443,97.149123
249,Onboarding Employee's IT-related Access and Ha...,25,4,35,0.008772,447,98.026316


Save the into excel file

In [ ]:
FilePath = r'C:\repo\SLA_Pareto_BeyondTAT.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(FilePath, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
newdf.to_excel(writer, sheet_name='Pareto Beyond TAT', index = False)
writer.save()